# Notebook to test how to use BERT and PyTorch with CUDA for FSL

Load dataset and transform data

In [1]:
import pandas as pd
import numpy as np

dataFrame = pd.read_pickle(r'../data/7587_corrige.pkl')
subset = dataFrame[['jobTitle', 'description', 'label']].copy()

subset.reset_index(drop=True, inplace=True)
subset.replace('', np.nan, inplace=True)
subset.dropna(inplace=True)

subset['text'] = subset['jobTitle'] + ' ' + subset['description']
subset = subset[['text','label']]
subset_label_transform = subset.copy()

subset_label_transform['label'] = np.where((subset_label_transform["label"] < 3) | (subset_label_transform["label"].isna()), 0, 1)
subset_label_transform

,text,label
2,Stagiaire ingénieur en intelligence artificiel...,1
3,Stagiaire en développement logiciel Développem...,0
4,Stagiaire en développement Web Création et évo...,0
5,Stagiaire en développement Web Portage d’une a...,0
6,Développeur Data / IA Développement d'applicat...,1
...,...,...
11281,Opérateur production Montage de transmission a...,0
11282,Opérateur production Montage de transmission a...,0
11283,Technicien réparation informatique Reparation ...,0
11284,Technicien réparation Reparation & maintenance...,0


Split between training and test set and create support set and load BERT

In [2]:
from utility import split_dataset
from protoNet import gen_support_set, get_tokenizer_and_model
from utility import get_n_shot_dataset
from datasets import Dataset

n_shots = 8

train_set, test_set = split_dataset(subset_label_transform, 0.2)
train_set = get_n_shot_dataset(train_set, n_shots)
train_set = Dataset.from_pandas(train_set)
test_set = Dataset.from_pandas(test_set)

tokenizer, model = get_tokenizer_and_model()

support_set = gen_support_set(n_shots, tokenizer, train_set)

Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

Training using prototypes

In [3]:
from protoNet import protonet_train, eval

model = protonet_train(support_set, train_set, tokenizer, model, num_epochs=20, batch_size=16, verbose=True)

Epoch:  1 / 20 ... Training loss: 0.50
Epoch:  2 / 20 ... Training loss: 0.49
Epoch:  3 / 20 ... Training loss: 0.50
Epoch:  4 / 20 ... Training loss: 0.48
Epoch:  5 / 20 ... Training loss: 0.49
Epoch:  6 / 20 ... Training loss: 0.48
Epoch:  7 / 20 ... Training loss: 0.47
Epoch:  8 / 20 ... Training loss: 0.45
Epoch:  9 / 20 ... Training loss: 0.44
Epoch:  10 / 20 ... Training loss: 0.45
Epoch:  11 / 20 ... Training loss: 0.40
Epoch:  12 / 20 ... Training loss: 0.37
Epoch:  13 / 20 ... Training loss: 0.32
Epoch:  14 / 20 ... Training loss: 0.30
Epoch:  15 / 20 ... Training loss: 0.24
Epoch:  16 / 20 ... Training loss: 0.25
Epoch:  17 / 20 ... Training loss: 0.23
Epoch:  18 / 20 ... Training loss: 0.27
Epoch:  19 / 20 ... Training loss: 0.20
Epoch:  20 / 20 ... Training loss: 0.21


Evaluation

In [4]:
f1_score = eval(test_set, tokenizer, model, support_set, verbose=True)
print(f"F1 score: {f1_score:.2f}")

Eval: 1 / 10
Eval: 2 / 10
Eval: 3 / 10
Eval: 4 / 10
Eval: 5 / 10
Eval: 6 / 10
Eval: 7 / 10
Eval: 8 / 10
Eval: 9 / 10
Eval: 10 / 10
F1 score: 0.31


Example

In [5]:
from protoNet import predict

pred1 = predict(tokenizer, model, "Mon rôle chez DreamQuark, est de résoudre les problématiques des différents acteurs autour de la\nbanque et assurance (Churn, upsale, cross-sale etc.) à travers des techniques de Machine\nLearning/Deep learning et analyse statistique.\n\n● Contribution à l'amélioration de Brain, la plateforme d'Auto-ML de Dreamquark, en développant de nouvelles features à l'aide du framework Pytorch, Scikit-learn, Numpy, Pandas, FastApi, Docker, Kubernetes et CircleCi\n\n● Développement d'un package Time Series avec l'intégration de module automatique de preprocessing et module de training avec des réseaux de neurone TCN (Temporal Convolutional Network)\n\n● Développement d'un moteur de data-preparation scalable à l'horizontal compatible Pandas et Dask, s'inspirant de la philosophe Pandas et scikit-learn pipeline permettant de rendre reproductible les codes jupyter en production.\n\nStack Technique :\n\nPython, Pytorch, Scikit-learn, Numpy, Docker, Kubernetes, Circleci, Dask, FastApi, Dask, Azure, Circle\nCi, Prefect, Alembic, SqlAlchemy, Postgresql'", support_set)
pred2 = predict(tokenizer, model, "• Utilisation de Flask et d’Elasticsearch afin de créer une API\nREST pour faire des recherches sur des régions de\nplanètes.\n\n• Conception d'une application web avec Vue.js et Quasar\nutilisant cette API, avec visualisation 3D des données.", support_set)

print("expected: ", [1,0], "predictions: ", [pred1,pred2])

expected:  [1, 0] predictions:  [1, 0]
